In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
import pyspark.sql
import pyspark.sql.types
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
from pyspark import SparkFiles
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_data = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True, timestampFormat="mm/dd/yy")
home_sales_data.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# 2. Create a temporary view of the DataFrame.
home_sales_data.createOrReplaceTempView('home_sales')
spark.sql("SELECT * FROM home_sales LIMIT 10").show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [14]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
    SELECT YEAR(date) as year,
            round(avg(price),2) as avg_price
    FROM home_sales
    WHERE bedrooms == 4
    GROUP BY YEAR(date)
    ORDER BY YEAR(date) ASC
    
"""
spark.sql(query).show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [15]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

query = """
    SELECT YEAR(date) as year,
            round(avg(price),2) as avg_price
    FROM home_sales
    WHERE bedrooms == 3 AND bathrooms = 3 
    GROUP BY YEAR(date)
    ORDER BY YEAR(date) ASC
    
"""
spark.sql(query).show()

+----+---------+
|year|avg_price|
+----+---------+
|2019|287287.82|
|2020|294204.16|
|2021|294211.46|
|2022|292725.69|
+----+---------+



In [17]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
    SELECT YEAR(date) as year,
            round(avg(price),2) as avg_price
    FROM home_sales
    WHERE bedrooms == 3 AND bathrooms = 3 AND floors == 2 AND sqft_living >= 2000
    GROUP BY YEAR(date)
    ORDER BY YEAR(date) ASC    
"""
spark.sql(query).show()


+----+---------+
|year|avg_price|
+----+---------+
|2019|289859.14|
|2020|292289.09|
|2021|296330.96|
|2022|290242.99|
+----+---------+



In [19]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query = """
    SELECT view,
            round(avg(price),2) as avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC   
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.28441739082336426 seconds ---


In [26]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [27]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [28]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """
    SELECT view,
            round(avg(price),2) as avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC   
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.24202513694763184 seconds ---


In [34]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales_data.write.partitionBy('date_built').parquet('home_sales_parquet')

AnalysisException: path file:/C:/Users/ducdu/OneDrive/Documents/UofT Bootcamp/Assignments/Big Data/home_sales_parquet already exists.

In [35]:
# 11. Read the formatted parquet data.
parquet_data=spark.read.parquet("home_sales_parquet")

Py4JJavaError: An error occurred while calling o121.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1218)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1423)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:225)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anon

In [36]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView('parquet_view')

NameError: name 'parquet_data' is not defined

In [37]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

query = """
    SELECT view,
            round(avg(price),2) as avg_price
    FROM parquet_view
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC   
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

AnalysisException: Table or view not found: parquet_view; line 4 pos 9;
'Sort ['view DESC NULLS LAST], true
+- 'Aggregate ['view], ['view, 'round('avg('price), 2) AS avg_price#1802]
   +- 'Filter ('price >= 350000)
      +- 'UnresolvedRelation [parquet_view], [], false


In [38]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [39]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False